In [11]:
! pip install 'git+https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git'

  Cloning https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-_m4m711w
  Running command git clone -q https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git /private/var/folders/lm/qgrqsck97r52_d1s83h76nkc0000gn/T/pip-req-build-_m4m711w
  Resolved https://github.com/nitkiew2/CMSE495-QSIDE-JUSTFAIR-TOOLBOX.git to commit c2d2e0d3fb790702c1e5547501a9590b3d98f942


In [12]:
import JUSTFAIR_Tools as jt
#from JUSTFAIR_Tools import toolbox

In [13]:
jt.toolbox.hello_world()

hello world!


In [14]:
jt.toolbox.matplotlib_test()

NameError: name 'plt' is not defined